In [3]:
import sys
sys.path.append('/home/xinyiz/pamrats')

import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim

# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score

import gae.gae.optimizer as optimizer
import gae.gae.model
import gae.gae.preprocessing as preprocessing

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN,MiniBatchKMeans,AgglomerativeClustering
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [14]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
ifplot=True
ifcluster=True
byCT=True

logitL1=0.5 #smaller is sparser
geneThresh=2.5 #times std above mean

# inverseAct=None
plottype='umap'
pca=PCA()
minCells=15 #min number of cells for analysis
minCell_clusterDE=5
# clustermethod=['leiden']
clustermethod=['leiden','agglomerative','kmeanbatch']
#umap/leiden clustering parameters
n_neighbors=10
min_dist=0.25
n_pcs=40 #for clustering
# resolution=[0.5,0.8,1,1.5]
resolution=[0.05,0.1,0.2,0.3,0.5,0.8,1,1.5]
plotepoch=9990
savenameAdd=''
#DBscan
epslist= [6,8,10]
min_sampleslist=[15,30,45] 
#agglomerative
nclusterlist=[2,3,4,5,8,10,15]
aggMetric=['euclidean']


combineCelltype={'glia':['Astro','Micro', 'OPC', 'Oligo'],'CA':['CA1', 'CA2', 'CA3']}

use_cuda=True
fastmode=False #Validate during training pass
seed=3

protein=None #'scaled_binary'
# proteinWeights=0.05
# randFeatureSubset=None
plot_samples={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
plot_sample_X=['raw']
plotRecon='' #'meanRecon'
# plot_sample_X=['corrected','scaled']
standardizeX=False
name='secretoryExprs'
plotsavepath='/mnt/external_ssd/xinyi/plots/data/'+name
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)
datadir='/home/xinyiz/2021-01-13-mAD-test-dataset'

In [5]:
#Load data
savedir=os.path.join('/mnt/xinyi/','starmap')
adj_dir=os.path.join(savedir,'a')

featureslist={}
if plot_sample_X[0] in ['corrected','scaled']:
    scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-scaled.h5ad')
    
    for s in plot_samples.keys():
        featureslist[s+'X_'+'corrected']=torch.tensor(scaleddata.layers['corrected'][scaleddata.obs['sample']==plot_samples[s]])
        featureslist[s+'X_'+'scaled']=torch.tensor(scaleddata.layers['scaled'][scaleddata.obs['sample']==plot_samples[s]])
    
else:
    scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-raw.h5ad')
    
    for s in plot_samples.keys():
        scaleddata_train=scaleddata.X[scaleddata.obs['sample']==plot_samples[s]]

        if plot_sample_X[0]=='logminmax':
            featurelog_train=np.log2(scaleddata_train+1/2)
            scaler = MinMaxScaler()
            featurelog_train_minmax=np.transpose(scaler.fit_transform(np.transpose(featurelog_train)))
            featureslist[s+'X_'+plot_sample_X[0]]=torch.tensor(featurelog_train_minmax)
        elif plot_sample_X[0]=='raw':
            featureslist[s+'X_'+plot_sample_X[0]]=scaleddata_train

In [48]:
pathway=pd.read_csv('/home/xinyiz/secretory/QuickGO-annotations-0032940-secretionByCell.tsv', sep='\t', header=0)

In [49]:
starmapG=np.array(scaleddata.var.index)

In [50]:
pathwayG=np.array(pathway['SYMBOL'].str.upper())

In [51]:
geneNames,commstarmap,commPathway=np.intersect1d(starmapG,pathwayG,return_indices=True)

In [54]:
np.random.seed(seed)
def plotExprs(exprs,vmax,savename,embedding,savepath,plotname,savenameAdd=''):
    fig, ax = plt.subplots(dpi=400)
    ax.scatter(embedding[:,0],embedding[:,1],c=exprs,cmap='Greys',vmin=0,vmax=vmax,edgecolors='none',s=1.5,alpha=1)
    plt.gca().set_aspect('equal', 'datalim')
    fig.set_figheight(5)
    fig.set_figwidth(5)
    plt.title(plotname, fontsize=18)
    plt.savefig(os.path.join(savepath,savename+savenameAdd+'.jpg'))
#     plt.show()
    
#     fig.clf()
    plt.close('all')

In [55]:
#all cells
vmax=0
for s in plot_samples.keys():
    sampleidx=plot_samples[s]
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        exprs=featureslist[samplename][:,commstarmap]
       
        vmax=max(vmax,np.max(np.sum(exprs,axis=1)))

for s in plot_samples.keys():
    sampleidx=plot_samples[s]
    
    celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
    celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
    region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        exprs=featureslist[samplename][:,commstarmap]
       
        savedir=plotsavepath

        plotExprs(np.sum(exprs,axis=1),vmax,'secretionByCell_'+s,sobj_coord_np,savedir,'secretion by cell '+s,savenameAdd=savenameAdd)
